## Transformation from Bronze to Silver
This script focus on :
- Cleaning the data from NULL values
- Converting the data format
- Standardizing the column names
- Create OrderDateCalendar dimension table

In [0]:
from pyspark.sql.functions import from_utc_timestamp, date_format, current_timestamp
from pyspark.sql.types import TimestampType
from pyspark.sql.functions import to_date, to_timestamp
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

In [0]:
#Cleaning and standardizing the data format of Order table and transfer to silver container

path = '/mnt/bronze/Orders/Orders.parquet'
df = spark.read.format('parquet').load(path)
df=df.filter((df['orderdate']!='NULL'))
df=df.filter((df['item_code']!='NULL'))
df=df.filter((df['CategoryNo']!='NULL'))
df = df.withColumn('orderdate', date_format(from_utc_timestamp(df['orderdate'].cast(TimestampType()), "UTC"), "yyyy-MM-dd"))
# df = df.withColumn("OrderDate",  date_format(to_date(df['orderdate'], "MM/dd/yy HH:mm"), "yyyy-MM-dd"))
df = df.withColumnRenamed("item_code", "ItemCode").withColumnRenamed("vendor_id", "VendorID").withColumnRenamed("orderdate", "OrderDate")
display(df)
output_path = '/mnt/silver/Orders/'
df.write.format('delta').mode("overwrite").option("mergeSchema", "true").save(output_path)


In [0]:
#Standardizing the column name of Vendor table and transfer to silver container

path = '/mnt/bronze/Vendor/Vendor.parquet'
df = spark.read.format('parquet').load(path)
df = df.withColumnRenamed("vendorname", "VendorName")
display(df)
output_path = '/mnt/silver/Vendor/'
df.write.format('delta').mode("overwrite").save(output_path)

In [0]:
#Cleaning and standardizing column name of OrderItem table and transfer to silver container

path = '/mnt/bronze/OrderItems/OrderItems.parquet'
df = spark.read.format('parquet').load(path)
df = df.withColumnRenamed("itemid", "ItemID").withColumnRenamed("item", "ItemDescription").withColumnRenamed("quantity", "Quantity").withColumnRenamed("itemno", "ItemNumber").withColumnRenamed("list_price", "ListPrice").withColumnRenamed("discount_price", "DiscountPrice").drop("subtotal")
display(df)
output_path = '/mnt/silver/OrderItem/'
df.write.format('delta').mode("overwrite").save(output_path)

In [0]:

#Transfer ItemCode table and transfer to silver container

path = '/mnt/bronze/ItemCode/ItemCode.parquet'
output_path = '/mnt/silver/ItemCode/'
df = spark.read.format('parquet').load(path)
df.write.format('delta').mode("overwrite").save(output_path)

In [0]:
#Create OrderDateCalendar table and transfer to silver container
from pyspark.sql import functions as F
from pyspark.sql.functions import *

df = spark.createDataFrame([(1,)], ["id"])

df1 = df.withColumn(
    "OrderDate", 
    F.explode(F.expr("sequence(to_date('2020-01-01'), to_date('2025-12-31'), interval 1 day)"))
)
output_path = '/mnt/silver/OrderDateCalendar/'
dforderdatecalendar = df1.withColumn("Year", year("OrderDate")).withColumn("Year", year("OrderDate")).withColumn("Month",month("OrderDate")).withColumn("Day",dayofmonth("OrderDate")).withColumn("Week", weekofyear("OrderDate")).withColumn("MonthName", date_format('OrderDate', 'MMMM')).withColumn("DayName", date_format('OrderDate', 'EEEE'))
dforderdatecalendar.write.format('delta').mode("overwrite").option("mergeSchema", "true").save(output_path)
dforderdatecalendar.show()